In [2]:
import tensorflow.keras as keras
import numpy as np

from keras.models import Model as KerasModel
from tensorflow.keras.preprocessing import image as keras_image
from keras.applications.imagenet_utils import decode_predictions, preprocess_input

def extract_representation_visaul(image_path, model_name, model_config, layer):
    
    Feature = db_feature(f"visual_representation-{model_name}_{layer}.db")
    
    # ============
    
    row = Feature.get_or_none(Feature.image == image_path)
    
    if row is not None:
        #print("Features is extracted: " + image_path)
        return np.array(eval(row.features), dtype=np.float32)
    
    # ============
    
    preprocess_visual(image_path)
    
    cropped_image_path = image_path + '.crop.jpg'
    
    # ============
    
    # model=keras.applications.convnext.ConvNeXtXLarge(weights="imagenet",include_top=True)
    model=eval(model_config)
    
    # ============
    
    img=keras_image.load_img(cropped_image_path,target_size=model.input_shape[1:3])
    x=keras_image.img_to_array(img)
    x=np.expand_dims(x,axis=0)
    # print(x)
    x=preprocess_input(x)
    # print(x)
    
    # ============
    
    # model.summary()
    # print(model_name)
    # print(layer)
    
    # feat_extractor=Model(inputs=model.input,outputs=model.get_layer("convnext_xlarge_head_gap").output[0])
    try:
        feat_extractor=KerasModel(inputs=model.input,outputs=model.get_layer(layer).output[0])
    except Exception as e:
        print("Error: " + model_name)
        raise e
        
    feat = feat_extractor.predict(x)
    # print(type(feat))
    
    feat_list = feat.flatten().tolist()
    # feat_list = [item for sublist in feat_list for item in sublist]

    # ============
    
    Feature(image=image_path, features=str(feat_list)).save()
    return feat
    

2024-05-31 08:08:14.020187: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-31 08:08:14.043639: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-31 08:08:14.043678: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-31 08:08:14.044419: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-31 08:08:14.048637: I tensorflow/core/platform/cpu_feature_guar